In [8]:
from functools import wraps
import time
import numpy as np
import unittest

# Utilities for timing
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__}{args} {kwargs} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [9]:
@timeit
def gen_traindata(num,Arange, eps, kappa, delta):
    N = num
    num_param = len(Arange)

    tau = np.linspace(0, 2 * np.pi, N)
    R_ellipse = np.zeros((N, num_param, num_param, num_param, num_param))
    Z_ellipse = np.zeros((N, num_param, num_param, num_param, num_param))
    A_ellipse = np.zeros((N, num_param, num_param, num_param, num_param))
    eps_ellipse = np.zeros((N, num_param, num_param, num_param, num_param))
    kappa_ellipse = np.zeros((N, num_param, num_param, num_param, num_param))
    delta_ellipse = np.zeros((N, num_param, num_param, num_param, num_param))
    for i in range(num_param):
        for j in range(num_param):
            for k in range(num_param):
                for kk in range(num_param):
                    R_ellipse[:, i, j, k, kk] = 1 + eps[j] * np.cos(tau + np.arcsin(delta[kk]) * np.sin(tau))
                    Z_ellipse[:, i, j, k, kk] = eps[j] * kappa[k] * np.sin(tau)
                    A_ellipse[:, i, j, k, kk] = Arange[i]
                    eps_ellipse[:, i, j, k, kk] = eps[j]
                    kappa_ellipse[:, i, j, k, kk] = kappa[k]
                    delta_ellipse[:, i, j, k, kk] = delta[kk]
    
    x_ellipse = np.transpose(
        np.asarray([
            R_ellipse, Z_ellipse, A_ellipse, 
            eps_ellipse, kappa_ellipse, delta_ellipse]), 
                             [1, 2, 3, 4, 5, 0])
    x_ellipse = x_ellipse.reshape(N * num_param ** 4, 6)
    uvals = np.zeros(len(x_ellipse)).reshape(len(x_ellipse), 1)
    return x_ellipse, uvals

@timeit
def gen_traindata_optimized(num,Arange, eps, kappa, delta):

    N = num
    tau = np.linspace(0, 2 * np.pi, N)

    # Generate a mesh grid of shape parameters and tau values
    A_ellipse, eps_ellipse, kappa_ellipse, delta_ellipse, tau = np.meshgrid(Arange, eps, kappa, delta, tau, indexing='ij')

    # Calculate R and Z values
    R = 1 + eps_ellipse* np.cos(tau + np.arcsin(delta_ellipse) * np.sin(tau))
    Z = eps_ellipse* kappa_ellipse * np.sin(tau)

    # Stack the arrays into a single output array
    output = np.stack((R,Z, A_ellipse, eps_ellipse, kappa_ellipse, delta_ellipse), axis=-1)

    # Reshape the output array to have shape (len(Arange) * len(eps) * len(kappa) * len(delta) * len(tau), 6)
    x_ellipse =output.reshape(-1, 6)
    uvals = np.zeros(len(x_ellipse)).reshape(len(x_ellipse), 1)
    return x_ellipse, uvals


In [16]:
class TestFunctions(unittest.TestCase):
    def test_functions(self):
        # Define input values
        Arange =  [-0.2, -0.1,  0.,   0.1 , 0.2] 
        eps =  [0.12, 0.22 ,0.32, 0.42, 0.52] 
        kappa =  [1. ,  1.35, 1.7,  2.05, 2.4 ] 
        delta =  [0.13, 0.23, 0.33 ,0.43, 0.53]
        num = 1000
        

        # Call the first function and save the output]
        x_ellipse, uvals= gen_traindata(num,Arange, eps, kappa, delta)
        sorted_x_ellipse = np.sort(np.sort(x_ellipse, axis=1), axis=0)

        # Call the second function and save the output
        x_ellipse_test, uvals_test =gen_traindata_optimized(num,Arange, eps, kappa, delta)
        sorted_x_ellipse_test = np.sort(np.sort(x_ellipse_test, axis=1), axis=0)

        # Compare the outputs using np.testing.assert_array_almost_equal
        np.testing.assert_array_almost_equal(sorted_x_ellipse, sorted_x_ellipse_test)

In [17]:
'''Main'''
# Create a test suite
test_suite = unittest.TestLoader().loadTestsFromTestCase(TestFunctions)

# Run the test suite
unittest.TextTestRunner(verbosity=2).run(test_suite)

test_functions (__main__.TestFunctions) ... 

Function gen_traindata(100000, [-0.2, -0.1, 0.0, 0.1, 0.2], [0.12, 0.22, 0.32, 0.42, 0.52], [1.0, 1.35, 1.7, 2.05, 2.4], [0.13, 0.23, 0.33, 0.43, 0.53]) {} Took 11.4555 seconds
Function gen_traindata_optimized(100000, [-0.2, -0.1, 0.0, 0.1, 0.2], [0.12, 0.22, 0.32, 0.42, 0.52], [1.0, 1.35, 1.7, 2.05, 2.4], [0.13, 0.23, 0.33, 0.43, 0.53]) {} Took 15.9925 seconds


KeyboardInterrupt: 